In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from itertools import product
from tqdm import tqdm_notebook
#from heuristi_plot_1 import plot as hplot

path = '../output_data/'

In [ ]:
with pd.HDFStore(path + 'X1a1.h5') as store:
    data = store.select('dat_1', where='phi=0.5 & n_rn>15', columns=['decision state', 'i_c', 'G'])
data.index = data.index.droplevel(['phi', 'test'])
data = data.reset_index('tstep')
data = data.reset_index().drop_duplicates(['n_rn', 'n_cp', 'sample', 'tstep']).set_index(['n_rn', 'n_cp', 'sample', 'tstep'])
data.index.levels

G_start = data[['G']].max()[0]

In [ ]:
c1 = 0.43 # tons of co2 per barrel oil https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references
c2 = 7.33 # barrels per toe https://en.wikipedia.org/wiki/Tonne_of_oil_equivalent
c3 = 10e6 # tons per megaton

# carbon budgets from 2011 onwards
b15p66 = 400 # Giga tones of co2 https://www.carbontracker.org/carbon-budgets-explained/ and IPCC2013
b15p50 = 550
b20p66 = 1000

x1 = b15p66/c1 # budged in giga tones CO2/CO2 emissions per barrel -> budget in billion barrels oil
x2 = x1/c2 # budget in billion barrels / barrels per toe -> budget in Gtoe
budget_mtoe = x2*10e3 # budget in Gtoe * 10^3 -> budget in Mtoe

# emissions per mtoe:
co2_per_toe = c1*c2 # tones of co2 per toe

data['emissions'] = (data[['G']].max() - data[['G']])*co2_per_toe*10e-4 # emissions in Gtones CO2

In [ ]:
# fraction of resource remaining at success of campaign

# when 2/3 vote for clean investment

data[['Gm']] = data[['G']]/G_start

data_tmp1 = data.loc[data['decision state']>.66][['Gm']].reset_index()

# for each pair of n_rn, n_cp and sample, select the row with smallest time (the one, where it tipped)
data_tmp2 = data_tmp1.loc[data_tmp1.groupby(['n_rn', 'n_cp', 'sample'])['tstep'].idxmin()]

# for that time, look at remaining resource and take the mean over it:
res1_mean = data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['Gm']].unstack('sample').mean(axis=1).unstack('n_cp')
res1_std = data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['Gm']].unstack('sample').std(axis=1).unstack('n_cp')

res1_min = data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['Gm']].unstack('sample').min(axis=1).unstack('n_cp')
res1_max = data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['Gm']].unstack('sample').max(axis=1).unstack('n_cp')

# and plot the fraction of remaining resource


fig, ax = plt.subplots(ncols=4, figsize=(25,5))

sn.heatmap(res1_mean, ax=ax[0])
sn.heatmap(res1_min, ax=ax[1])
sn.heatmap(res1_max, ax=ax[2])
sn.heatmap(res1_std, ax=ax[3])

for ax, text in zip(ax, ['fraction of', 'min fraction of', 'max fraction of', 'std of']):
    ax.invert_yaxis()
    txt = ax.set_title(f'{text} resource remaining')

In [ ]:
# fraction of resource remaining in 2050

data_tmp1 = data[['Gm']].xs(level='tstep', key=90).unstack('sample')

res2_mean = data_tmp1.mean(axis=1).unstack('n_cp')
res2_min = data_tmp1.min(axis=1).unstack('n_cp')
res2_max = data_tmp1.max(axis=1).unstack('n_cp')
res2_std = data_tmp1.std(axis=1).unstack('n_cp')

fig, ax = plt.subplots(ncols=4, figsize=(25,5))

sn.heatmap(res2_mean, ax=ax[0])
sn.heatmap(res2_min, ax=ax[1])
sn.heatmap(res2_max, ax=ax[2])
sn.heatmap(res2_std, ax=ax[3])

for ax, text in zip(ax, ['fraction of', 'min fraction of', 'max fraction of', 'std of']):
    ax.invert_yaxis()
    txt = ax.set_title(f'{text} resource remaining in 2090')

In [ ]:
# probability of staying within the 1.5C, p=66% budget at the sucess of the campaign

# when 2/3 vote for clean investment

data_tmp1 = data.loc[data['decision state']>.66][['emissions']].reset_index()

# for each pair of n_rn, n_cp and sample, select the row with smallest time (the one, where it tipped)
data_tmp2 = data_tmp1.loc[data_tmp1.groupby(['n_rn', 'n_cp', 'sample'])['tstep'].idxmin()]

(data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['emissions']]<b15p66).unstack('sample').mean(axis=1)

# for that time, look at remaining resource and take the mean over it:
res1_mean = (data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['emissions']]<b15p66).unstack('sample').mean(axis=1).unstack('n_cp')
res1_std = (data_tmp2.set_index(['n_rn', 'n_cp', 'sample'])[['emissions']]<b15p66).unstack('sample').std(axis=1).unstack('n_cp')

# and plot the fraction of remaining resource

fig1, ax1 = plt.subplots(ncols=2, figsize=(12,5))

sn.heatmap(res1_mean, ax=ax1[0])
sn.heatmap(res1_std, ax=ax1[1])

ax1[0].set_title('probability of staying under the 1.5 budget when 2/3 vote clean')
ax1[1].set_title('standard error of the mean')
for axis in ax1:
    axis.invert_yaxis()

In [ ]:
# probability of staying within the 1.5C, p=66% budget at year 2010 + t_max

# when 2/3 vote for clean investment
t_max = 25
target = b15p50

data_tmp1 = data.xs(level='tstep', key=t_max)[['emissions']].reset_index()

# for that time, look at remaining resource and take the mean over it:
res2_mean = (data_tmp1.set_index(['n_rn', 'n_cp', 'sample'])[['emissions']]<target).unstack('sample').mean(axis=1).unstack('n_cp')
res2_std = (data_tmp1.set_index(['n_rn', 'n_cp', 'sample'])[['emissions']]<target).unstack('sample').std(axis=1).unstack('n_cp')

# and plot the fraction of remaining resource

fig2, ax2 = plt.subplots(ncols=2, figsize=(12,5))

sn.heatmap(res2_mean, ax=ax2[0])
sn.heatmap(res2_std, ax=ax2[1])

ax2[0].set_title(f'probability of staying under the 1.5 budget in {2010+t_max}')
ax2[1].set_title('standard error of the mean')
for axis in ax2:
    axis.invert_yaxis()
print('Why is the standard error of the mean so high?!')

In [ ]:
fig, ax = plt.subplots()

rednecks = [16, 25, 30]
colors = ['green', 'orange', 'red']

for c, rn in zip(colors, rednecks):

    df = res2_mean.stack('n_cp').xs(level='n_rn', key=rn)
    df_s = res2_std.stack('n_cp').xs(level='n_rn', key=rn)
    x = df.index.values
    y = df.values
    dy = df_s.values

    ax.plot(x, y, '.', color=c)
    ax.fill_between(x, y+dy, y-dy, alpha=.5, color=c)

ax.set_ylim([0,1])

In [ ]:
data_tmp1 = data.xs(level='tstep', key=40)[['emissions']].reset_index()
data_tmp1

In [ ]:
res2 = pd.DataFrame(index = ix2)

co2_per_oil = 3.5
initial_resource = data['G'].max()
budget = 886*10e2

for i, j, k in tqdm_notebook(list(product(a, b, c))):
    
    tmp = data.xs(level=('n_rn', 'n_cp', 'sample'), key=(i, j, k)).reset_index()[['decision state', 'tstep', 'G']]
    
    t = None
    for _, row in tmp.iterrows():
        budget_used = (initial_resource - row['G'])*co2_per_oil
        #print(row['tstep'], budget_used, budget)
        if budget_used > budget and t is None:
            t = row['tstep']
    if t is not None:
        if t < 5:
            t = None
    res2.loc[(i, j, k), 't'] = t
    res2.loc[(i, j, k), 'G'] = G
res1.to_pickle('res2.pkl')

In [ ]:
sn.heatmap(res2['t'].unstack('sample').std(axis=1).unstack('n_cp'))

In [ ]:
data = data.reset_index()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12,5))
data['G'].unstack(['n_rn', 'n_cp', 'sample']).plot(legend=False,
                                                  ax=axes[0],
                                                  color='k',
                                                  alpha=0.2)

In [ ]:
data['G'].max()*(.7)

In [ ]:
co2_per_oil = 3.5
initial_resource = data['G'].max()

In [ ]:
initial_resource

In [ ]:
G = data.xs(level=('n_rn', 'n_cp', 'sample'), key=(18, 0, 0))[['G']]

In [ ]:
tmp = G[0:1].values
tmp[0]-tmp[1]